In [1]:
import os
import json
from PIL import Image, ImageDraw
import struct
import binascii
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
items = [f[:-4] for f in os.listdir('Raw') if f[-4:]=='.bmp']

# check if table bbox exists
valid_items = []
for item in items:
    with open('Labeled/'+item+'.xml', "r") as f:
        xml = f.read()
        if xml.find('Composites Label="Table"')!=-1:
            valid_items.append(item)

In [3]:
for item in valid_items:
    width, height = Image.open('Evaluated_Images/'+item+'.jpg').size
    im = Image.open('Evaluated_Images/'+item+'.jpg').convert('RGB')
    with open('Labeled/'+item+'.xml', "r") as f:
        xml = f.read()
        table_block = xml.split('Composites Label="Table"')[1].split('</Composites>')[0] # store all tables
        samples = table_block.split('<Composite BBox="')[1:]
        for sample in samples:
            bbox_hex = sample.split('"')[0].split() # wannable bbox (hex string)
            bbox_int = [round(96/72*struct.unpack('!d', bytes.fromhex(x))[0]) for x in bbox_hex] #from ieee754 to int
            xmin, ymin, xmax, ymax = bbox_int
            ymin = height - ymin
            ymax = height - ymax
            draw = ImageDraw.Draw(im, 'RGBA')
            draw.line((xmin, ymin) + (xmin, ymax), fill=(255, 80, 80, 100), width=4)
            draw.line((xmin, ymin) + (xmax, ymin), fill=(255, 80, 80, 100), width=4)
            draw.line((xmax, ymax) + (xmin, ymax), fill=(255, 80, 80, 100), width=4)
            draw.line((xmax, ymax) + (xmax, ymin), fill=(255, 80, 80, 100), width=4)
            del draw
        im.save('Evaluated_GT_images/'+item+'.jpg')

In [15]:
# obtaining ground truths: label left top right bottom 
for item in valid_items:
    width, height = Image.open('Evaluated_Images/'+item+'.jpg').size
    with open('Labeled/'+item+'.xml', "r") as f:
        xml = f.read()
        table_block = xml.split('Composites Label="Table"')[1].split('</Composites>')[0] # store all tables
        samples = table_block.split('<Composite BBox="')[1:]
        with open('Ground_Truth/'+item+'.txt', "w") as f_out:
            for sample in samples:
                bbox_hex = sample.split('"')[0].split() # wannable bbox (hex string)
                bbox_int = [round(96/72*struct.unpack('!d', bytes.fromhex(x))[0]) for x in bbox_hex] #from ieee754 to int
                xmin, ymin, xmax, ymax = bbox_int
                ymin = height - ymin
                ymax = height - ymax
                (left, right, top, bottom) = (xmin, xmax, ymin, ymax)
                f_out.write(' '.join(['table', str(left), str(top), str(right), str(bottom)]))
                f_out.write('\n')